In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pywt import wavedec
import pandas as pd
from scipy.fftpack import fft,ifft
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostRegressor
import joblib
import pywt.data
import pickle as pickle
import random
import math
from sklearn import svm
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from minepy import MINE

In [2]:
movieList=['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39']
movieList_int=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39]
channel=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32]#前32个通道为脑电信号
subjectList= ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']
#subjectList= ['01','02','03','04','05','06','07','08']
band = [4,8,12,16,25,45] #5 bands
window_size = 128 #Averaging band power of 2 sec
step_size = 128 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
movielenth=7680
piecesNumber=((movielenth-window_size)/step_size)+1

In [191]:
def bin_power(X, Band, Fs):
    

    C = np.fft.fft(X)
    C = abs(C)
    Power = np.zeros(len(Band) - 1)
    for Freq_Index in range(0, len(Band) - 1):
        Freq = float(Band[Freq_Index])
        Next_Freq = float(Band[Freq_Index + 1])
        Power[Freq_Index] = sum(
            C[int(np.floor(Freq / Fs * len(X))): 
                int(np.floor(Next_Freq / Fs * len(X)))]
        )
    Power_Ratio = Power / sum(Power)
    return Power, Power_Ratio

def FFT_Processing_subbasefeature_selectchannel_firstO(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for t in range(3):
                    tmp = 0
                    bas=base[t]
                    for k in range(0,len(bas)-1):
                        tmp = tmp + abs(bas[k+1] - bas[k] )
                    Y = tmp / (len(bas)-1)
                    basefeature.append(Y)
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
            #print(np.array(allChannelbase).shape)
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    tmp = 0
                    for k in range(0,len(X)-1):
                        tmp = tmp + abs(X[k+1] - X[k] )
                    Y = tmp / (len(X)-1)
                    meta_data = meta_data + list(([Y]-allChannelbase[num]))
                    num=num+1
                    #print(np.array(meta_data).shape)
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)
        np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True) 
        

def FFT_Processing_subbasefeature_selectchannel_kurt(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for k in range(3):
                    tmp = 0
                    Y=pd.Series(base[k]).kurt()
                    basefeature.append(Y)
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
            #print(np.array(allChannelbase).shape)
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y=pd.Series(X).kurt()
                    meta_data = meta_data + list(([Y]-allChannelbase[num]))
                    num=num+1
                    
                    #print(np.array(meta_data).shape)
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)
        np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True) 
def FFT_Processing_subbasefeature_selectchannel_skew(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for k in range(3):
                    tmp = 0
                    Y=pd.Series(base[k]).skew()
                    basefeature.append(Y)
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
            #print(np.array(allChannelbase).shape)
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y=pd.Series(X).kurt()
                    meta_data = meta_data + list(([Y]-allChannelbase[num]))
                    num=num+1
                    #print(np.array(meta_data).shape)
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)
        np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True) 
def FFT_Processing_subbasefeature_selectchannel_var(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for k in range(3):
                    tmp = 0
                    Y=np.var(base[k])
                    basefeature.append(Y)
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
            #print(np.array(allChannelbase).shape)
        
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y=np.var(X)
                    meta_data = meta_data + list(([Y]-allChannelbase[num]))
                    num=num+1
                    #print(np.array(meta_data).shape)
                    
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)
        np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True) 
def FFT_Processing_subbasefeature_selectchannel_secondO(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for t in range(3):
                    tmp = 0
                    bas=base[t]
                    for k in range(0,len(bas)-2):
                        tmp = tmp + abs(bas[k+2] - 2 * bas[k+1] + bas[k] )
                    Y = tmp / (len(bas)-2)
                    basefeature.append(Y)
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
                
            #print(np.array(allChannelbase).shape)
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec

                    tmp = 0
                    for k in range(0,len(X)-2):
                        tmp = tmp + abs(X[k+2] - 2 * X[k+1] + X[k] )
                    Y = tmp / (len(X)-2)
                    #print([Y]-allChannelbase[num])
                    meta_data = meta_data + list(([Y]-allChannelbase[num]))
                    num=num+1
                    
                    #print(np.array(meta_data).shape)
                meta_array.append(np.array(meta_data))
                
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)
        np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True) 



def waveAllEngergy(arr1,arr2,arr3,arr4,arr5):
    SUM=sum([waveEngergy(arr1),waveEngergy(arr2),waveEngergy(arr3),waveEngergy(arr4),waveEngergy(arr5)])
    return SUM

def waveEngergy(engergy):
    result=sum([x ** 2 for x in engergy])
    return result

def waveEntropy(en):
    result=-en*np.log2(en)
    return result

def FFT_Processing_subbasefeature_selectchannel_waven(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for k in range(3):
                    A4,D4,D3,D2,D1 = wavedec(base[k], 'db4',level=4)
                    #求相对小波能量
                    fp1=waveEngergy(A4)/waveAllEngergy(A4,D4,D3,D2,D1)
                    fp2=waveEngergy(D4)/waveAllEngergy(A4,D4,D3,D2,D1)
                    fp3=waveEngergy(D3)/waveAllEngergy(A4,D4,D3,D2,D1)
                    fp4=waveEngergy(D2)/waveAllEngergy(A4,D4,D3,D2,D1)
                    fp5=waveEngergy(D1)/waveAllEngergy(A4,D4,D3,D2,D1)
                    mean = []
                    mean.append(waveEntropy(fp1))
                    mean.append(waveEntropy(fp2))
                    mean.append(waveEntropy(fp3))
                    mean.append(waveEntropy(fp4))
                    mean.append(waveEntropy(fp5))
                    basefeature.append(mean)
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
            #print(np.array(allChannelbase).shape)
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    A4,D4,D3,D2,D1 = wavedec(X, 'db4',level=4)
                    #求相对小波能量
                    fp1=waveEngergy(A4)/waveAllEngergy(A4,D4,D3,D2,D1)
                    fp2=waveEngergy(D4)/waveAllEngergy(A4,D4,D3,D2,D1)
                    fp3=waveEngergy(D3)/waveAllEngergy(A4,D4,D3,D2,D1)
                    fp4=waveEngergy(D2)/waveAllEngergy(A4,D4,D3,D2,D1)
                    fp5=waveEngergy(D1)/waveAllEngergy(A4,D4,D3,D2,D1)

                    mean = []
                    mean.append(fp1)
                    mean.append(fp2)
                    mean.append(fp3)
                    mean.append(fp4)
                    mean.append(fp5)

                    meta_data = meta_data + list(mean-allChannelbase[num])
                    num=num+1
                    #print(np.array(meta_data).shape)
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)
        np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True) 
        

#能量谱
def engergy(X, Band, Fs):
    C = np.fft.fft(X)
    C = abs(C)
    en = np.zeros(len(Band) - 1)
    for Freq_Index in range(0, len(Band) - 1):
        Freq = float(Band[Freq_Index])
        Next_Freq = float(Band[Freq_Index + 1])
        fred_band=C[int(np.floor(Freq / Fs * len(X))): int(np.floor(Next_Freq / Fs * len(X)))]
#         Power[Freq_Index] = sum(index*index for index in fred_band)
        en[Freq_Index] = sum(index*index for index in fred_band)
    return en

def de_Entropy(X, Band, Fs):
    C = np.fft.fft(X)
    C = abs(C)
    en = np.zeros(len(Band) - 1)
    for Freq_Index in range(0, len(Band) - 1):
        Freq = float(Band[Freq_Index])
        Next_Freq = float(Band[Freq_Index + 1])
        fred_band=C[int(np.floor(Freq / Fs * len(X))): int(np.floor(Next_Freq / Fs * len(X)))]
#         Power[Freq_Index] = sum(index*index for index in fred_band)
        en[Freq_Index] = sum(index*index for index in fred_band)
    return np.log(en)

def FFT_Processing_subbasefeature_selectchannel_de_Entropy(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + subjects + '.dat', 'rb') as file:
        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for k in range(3):
                    #Power_Ratio= mybin_power(base[k], band, sample_rate)
                    #Y =pyeeg.spectral_entropy(base[k], band, sample_rate,Power_Ratio)
                    Y =  de_Entropy(base[k],band,sample_rate)
                    #Y = bin_power(base[k], band, sample_rate)
                    basefeature.append(Y)
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
            #print(np.array(allChannelbase).shape)
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y =  de_Entropy(X,band,sample_rate)
                    
                    meta_data = meta_data + list((Y)-allChannelbase[num])


                    num=num+1
                    #print(np.array(meta_data).shape)
                
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)
        np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True)
        
def FFT_Processing_subbasefeature_selectchannel_engergy(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + subjects + '.dat', 'rb') as file:
        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for k in range(3):
                    #Power_Ratio= mybin_power(base[k], band, sample_rate)
                    #Y =pyeeg.spectral_entropy(base[k], band, sample_rate,Power_Ratio)
                    Y =  engergy(base[k],band,sample_rate)
                    #Y = bin_power(base[k], band, sample_rate)
                    basefeature.append(Y)
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
            #print(np.array(allChannelbase).shape)
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y =  engergy(X,band,sample_rate)
                    
                    meta_data = meta_data + list((Y)-allChannelbase[num])


                    num=num+1
                    #print(np.array(meta_data).shape)
                
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)
        np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True)
def FFT_Processing_subbasefeature_selectchannel_newfeature_computetime(sub, channel,band, window_size, step_size, sample_rate):
    meta_de = []
    with open('data\s' + subjects + '.dat', 'rb') as file:
        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for k in range(3):
                    #Power_Ratio= mybin_power(base[k], band, sample_rate)
                    #Y =pyeeg.spectral_entropy(base[k], band, sample_rate,Power_Ratio)
                    Y = bin_power(base[k], band, sample_rate)
                    basefeature.append(Y[1]*np.log2(Y[1]))
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
            #print(np.array(allChannelbase).shape)
            
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    #Power_Ratio= mybin_power(X, band, sample_rate)
                    #Y =pyeeg.spectral_entropy(X, band, sample_rate,Power_Ratio)
                    #meta_data = meta_data +[Y-allChannelbase[num]]
                    meta_data = meta_data + list((Y[1]*np.log2(Y[1]))-allChannelbase[num])


                    num=num+1
                    #print(np.array(meta_data).shape)
                
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)
        #np.save('log2\s' + sub, meta_de, allow_pickle=True, fix_imports=True)

        
def FFT_Processing_subbasefeature_selectchannel_mean(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for k in range(3):
                    Y = np.mean(base[k])
                    basefeature.append(Y)
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
            #print(np.array(allChannelbase).shape)
            
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y=np.mean(X) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    #print((np.log2(Y[0]).shape))
                    meta_data = meta_data + list([Y]-allChannelbase[num])
                    num=num+1
                    #print(np.array(meta_data).shape)
                    
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)

        np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True) 
def FFT_Processing_subbasefeature_selectchannel_std(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for t in range(3):
                    tmp = 0
                    bas=base[t]
                    for k in range(0,len(bas)-1):
                        tmp = tmp + abs(bas[k+1] - bas[k])
                    Y = tmp / (len(bas)-1)
                    Y=Y/np.std(bas);
                    basefeature.append(Y)
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
           #print(np.array(allChannelbase).shape)
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    tmp = 0
                    
                    for k in range(0,len(X)-1):
                        tmp = tmp + abs(X[k+1] - X[k])
                        
                    Y = tmp / (len(X)-1)
                    Y=Y/np.std(X)
                    
                    meta_data = meta_data + list(([Y]-allChannelbase[num]))
                    #print(np.array(meta_data).shape)
                    
                    num=num+1
                    #print(np.array(meta_data).shape)
              
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)

        np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True) 

In [177]:
for path2 in range(0,2):
    for subjects in subjectList:
        
        FFT_Processing_subbasefeature_selectchannel_std (subjects, channel, band, window_size, step_size, sample_rate, 'std', path2)
        FFT_Processing_subbasefeature_selectchannel_secondO (subjects, channel, band, window_size, step_size, sample_rate, 'secondO', path2)
        FFT_Processing_subbasefeature_selectchannel_firstOm (subjects, channel, band, window_size, step_size, sample_rate, 'firstOm', path2)

In [79]:
for path2 in range(0,2):
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_wav (subjects, channel, band, window_size, step_size, sample_rate, 'wav', path2)
        FFT_Processing_subbasefeature_selectchannel_mean (subjects, channel, band, window_size, step_size, sample_rate, 'mean', path2)
        FFT_Processing_subbasefeature_selectchannel_newfeature_computetime(subjects, channel, band, window_size, step_size, sample_rate)
        FFT_Processing_subbasefeature_selectchannel_skew (subjects, channel, band, window_size, step_size, sample_rate, 'skew', path2)
        FFT_Processing_subbasefeature_selectchannel_var (subjects, channel, band, window_size, step_size, sample_rate, 'var', path2)
        FFT_Processing_subbasefeature_selectchannel_firstOm (subjects, channel, band, window_size, step_size, sample_rate, 'firstOm', path2)
        FFT_Processing_subbasefeature_selectchannel_secondO (subjects, channel, band, window_size, step_size, sample_rate, 'secondO', path2)
        FFT_Processing_subbasefeature_selectchannel_de_Entropy(subjects, channel, band, window_size, step_size, sample_rate, 'de_Entropy', path2)
        FFT_Processing_subbasefeature_selectchannel_engergy(subjects, channel, band, window_size, step_size, sample_rate, 'engergy', path2)
        FFT_Processing_subbasefeature_selectchannel_kurt (subjects, channel, band, window_size, step_size, sample_rate, 'kurt', path2)
        FFT_Processing_subbasefeature_selectchannel_std (subjects, channel, band, window_size, step_size, sample_rate, 'std', path2)

In [182]:
def FFT_Processing_subbasefeature_selectchannel_psifeature(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + subjects + '.dat', 'rb') as file:
        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:#0A1V
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for k in range(3):
                    #Power_Ratio= mybin_power(base[k], band, sample_rate)
                    #Y =pyeeg.spectral_entropy(base[k], band, sample_rate,Power_Ratio)
                    Y = bin_power(base[k], band, sample_rate)
                    basefeature.append(Y[0])
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
            #print(np.array(allChannelbase).shape)
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    #Power_Ratio= mybin_power(X, band, sample_rate)
                    #Y =pyeeg.spectral_entropy(X, band, sample_rate,Power_Ratio)
                    #meta_data = meta_data +[Y-allChannelbase[num]]
                    meta_data = meta_data + list(Y[0]-allChannelbase[num])


                    num=num+1
                    #print(np.array(meta_data).shape)
                
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)
        np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True) 
        
def FFT_Processing_subbasefeature_selectchannel_rirfeature(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + subjects + '.dat', 'rb') as file:
        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:#0A1V
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for k in range(3):
                    #Power_Ratio= mybin_power(base[k], band, sample_rate)
                    #Y =pyeeg.spectral_entropy(base[k], band, sample_rate,Power_Ratio)
                    Y = bin_power(base[k], band, sample_rate)
                    basefeature.append(Y[1])
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
            #print(np.array(allChannelbase).shape)
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    #Power_Ratio= mybin_power(X, band, sample_rate)
                    #Y =pyeeg.spectral_entropy(X, band, sample_rate,Power_Ratio)
                    #meta_data = meta_data +[Y-allChannelbase[num]]
                    meta_data = meta_data + list(Y[1]-allChannelbase[num])


                    num=num+1
                    #print(np.array(meta_data).shape)
                
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)
        np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True) 
        #np.save('logcontrast\crir\c1\s' + sub, meta_de, allow_pickle=True, fix_imports=True)

In [194]:
#二次验证
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_newfeature_computetime(subjects, channel, band, window_size, step_size, sample_rate)
    elapsed = (time.clock() - start)
    print(elapsed)

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


98.55128699995112
98.95964119990822


In [197]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_psifeature(subjects, channel, band, window_size, step_size, sample_rate, 'psi', path2)
    elapsed = (time.clock() - start)
    print(elapsed)

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


103.36733879998792
99.13749090000056


In [196]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_rirfeature(subjects, channel, band, window_size, step_size, sample_rate, 'rir', path2)
    elapsed = (time.clock() - start)
    print(elapsed)

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


97.14927329996135
102.27122550003696


In [198]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_waven(subjects, channel, band, window_size, step_size, sample_rate, 'waven', path2)
    elapsed = (time.clock() - start)
    print(elapsed)

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


1479.2211471999763
1469.2222443000646


In [31]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_wav (subjects, channel, band, window_size, step_size, sample_rate, 'wav', path2)
    elapsed = (time.clock() - start)
    print(elapsed)


D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


1323.3524413000005
1325.0179961000003


In [12]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_mean (subjects, channel, band, window_size, step_size, sample_rate, 'mean', path2)
    elapsed = (time.clock() - start)
    print(elapsed)

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


20.406012200000077
19.896240800000214


In [22]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_newfeature_computetime(subjects, channel, band, window_size, step_size, sample_rate)
    elapsed = (time.clock() - start)
    print(elapsed)

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


85.0796749000001
85.42700369999966


In [14]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_skew (subjects, channel, band, window_size, step_size, sample_rate, 'skew', path2)
    elapsed = (time.clock() - start)
    print(elapsed)

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


327.29057409999996
326.88688490000004


In [15]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_var (subjects, channel, band, window_size, step_size, sample_rate, 'var', path2)
    elapsed = (time.clock() - start)
    print(elapsed)

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


48.34474380000029
49.59791559999985


In [17]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_firstOm (subjects, channel, band, window_size, step_size, sample_rate, 'firstOm', path2)
    elapsed = (time.clock() - start)
    print(elapsed)

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


236.2063103999999
265.59714250000025


In [23]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_secondO (subjects, channel, band, window_size, step_size, sample_rate, 'secondO', path2)
    elapsed = (time.clock() - start)
    print(elapsed)


D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


237.3756696000005
242.3663421000001


In [25]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
         FFT_Processing_subbasefeature_selectchannel_de_Entropy(subjects, channel, band, window_size, step_size, sample_rate, 'de_Entropy', path2)
    elapsed = (time.clock() - start)
    print(elapsed)

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


93.46899209999992
95.42163470000014


In [26]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_engergy(subjects, channel, band, window_size, step_size, sample_rate, 'engergy', path2)
    elapsed = (time.clock() - start)
    print(elapsed)

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


98.57124820000081
95.63695610000013


In [27]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_kurt (subjects, channel, band, window_size, step_size, sample_rate, 'kurt', path2)
    elapsed = (time.clock() - start)
    print(elapsed)     

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


331.84152559999984
331.0095406999999


In [34]:
import time
for path2 in range(0,2):
    start = time.clock()
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_std (subjects, channel, band, window_size, step_size, sample_rate, 'std', path2)
    elapsed = (time.clock() - start)
    print(elapsed)

D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


359.090139599999
351.02651049999986


In [179]:
random_list_Test=[]
random_list_Validation=[]
random_list_Train=[]
with open('log1\list_train'+'.npy', 'rb') as fileTrain:
    random_list_Train  = np.load(fileTrain)
with open('log1\list_test'+'.npy', 'rb') as fileTrain:
    random_list_Test  = np.load(fileTrain)
with open('log1\list_validation'+'.npy', 'rb') as fileTrain:
    random_list_Validation  = np.load(fileTrain)
#print("测试集",random_list_Test)
#print("验证集",random_list_Validation)
#print("训练集",random_list_Train)
namelist=['de_Entropy','engergy','firstOm','kurt','mean','secondO','skew','std','var','wav']

for path2 in range(0,2):
    for path1 in namelist:
        acc_v1 = []
        for subjects in subjectList:
            dataTest=[]
            labelTest=[]
            dataValidation=[]
            labelValidation=[]
            dataTrain=[]
            labelTrain=[]
            dataTestValidation=[]
            labelTestValidation=[]

            with open('logcontrastv\c'+str(path1)+'\c'+str(path2)+'\s' + subjects + '.npy', 'rb') as file:
                sub = np.load(file, allow_pickle=True)
                for i in range (0,sub.shape[0]):
                    if i in random_list_Train:
                        dataTrain.append(np.ravel(sub[i][0]))
                        labelTrain.append(sub[i][1])
                    if i in random_list_Test:
                        dataTest.append(np.ravel(sub[i][0]))
                        labelTest.append(sub[i][1])
                    if i in random_list_Validation:
                        dataValidation.append(np.ravel(sub[i][0]))
                        labelValidation.append(sub[i][1])

            #print("subjects:", subjects)

            #print('loading data...')
            X  = np.array(dataTrain)
            Y  = np.array(labelTrain)
            X = normalize(X)



            M = np.array(dataTest)
            N = np.array(labelTest)
            M = normalize(M)



            O = np.array(dataValidation)
            P = np.array(labelValidation)
            O = normalize(O)
            np.save('logcontrastv\c'+str(path1)+'\c'+str(path2)+'\data_testing'+str(subjects)+'.npy', np.array(dataTest), allow_pickle=True, fix_imports=True)
            np.save('logcontrastv\c'+str(path1)+'\c'+str(path2)+'\label_testing'+str(subjects)+'.npy', np.array(labelTest), allow_pickle=True, fix_imports=True)
            np.save('logcontrastv\c'+str(path1)+'\c'+str(path2)+'\data_validationing'+str(subjects)+'.npy', np.array(dataValidation), allow_pickle=True, fix_imports=True)
            np.save('logcontrastv\c'+str(path1)+'\c'+str(path2)+'\label_validationing'+str(subjects)+'.npy', np.array(labelValidation), allow_pickle=True, fix_imports=True)
            np.save('logcontrastv\c'+str(path1)+'\c'+str(path2)+'\data_training'+str(subjects)+'.npy', np.array(dataTrain), allow_pickle=True, fix_imports=True)
            np.save('logcontrastv\c'+str(path1)+'\c'+str(path2)+'\label_training'+str(subjects)+'.npy', np.array(labelTrain), allow_pickle=True, fix_imports=True)


            train_size = np.array(X).shape[0]
            #print(np.array(X).shape[0],"------train_size------",np.array(X).shape[1])


            #print("-----------svm1分类----------")
            SV = svm.SVC(kernel='rbf')
            SV.fit(X, Y)
            acc_matrix=SV.score(M, N)
            acc_v1.append(acc_matrix)

            #print(acc_matrix)


        acc_v_mean1 = np.mean(acc_v1)
        #for i in range(len(subjectList)):
            #print("subjects[",i,"]")
            #print("acc_v_mean:",acc_v1[i])
        print("------------")
        print("path:",path1,path2)
        print("acc_v_mean:",acc_v_mean1)

------------
path: de_Entropy 0
acc_v_mean: 0.87984375
------------
path: engergy 0
acc_v_mean: 0.723828125
------------
path: firstOm 0
acc_v_mean: 0.78046875
------------
path: kurt 0
acc_v_mean: 0.760234375
------------
path: mean 0
acc_v_mean: 0.60109375
------------
path: secondO 0
acc_v_mean: 0.8025
------------
path: skew 0
acc_v_mean: 0.66
------------
path: std 0
acc_v_mean: 0.74125
------------
path: var 0
acc_v_mean: 0.6723437499999999
------------
path: wav 0
acc_v_mean: 0.9009375
------------
path: de_Entropy 1
acc_v_mean: 0.881640625
------------
path: engergy 1
acc_v_mean: 0.761484375
------------
path: firstOm 1
acc_v_mean: 0.787109375
------------
path: kurt 1
acc_v_mean: 0.77796875
------------
path: mean 1
acc_v_mean: 0.6553906249999999
------------
path: secondO 1
acc_v_mean: 0.807890625
------------
path: skew 1
acc_v_mean: 0.682734375
------------
path: std 1
acc_v_mean: 0.7633593750000001
------------
path: var 1
acc_v_mean: 0.7022656250000001
------------
path: 

In [199]:
random_list_Test=[]
random_list_Validation=[]
random_list_Train=[]
with open('log1\list_train'+'.npy', 'rb') as fileTrain:
    random_list_Train  = np.load(fileTrain)
with open('log1\list_test'+'.npy', 'rb') as fileTrain:
    random_list_Test  = np.load(fileTrain)
with open('log1\list_validation'+'.npy', 'rb') as fileTrain:
    random_list_Validation  = np.load(fileTrain)
#print("测试集",random_list_Test)
#print("验证集",random_list_Validation)
#print("训练集",random_list_Train)
namelist=['waven','rir','psi']

for path2 in range(0,2):
    for path1 in namelist:
        acc_v1 = []
        for subjects in subjectList:
            dataTest=[]
            labelTest=[]
            dataValidation=[]
            labelValidation=[]
            dataTrain=[]
            labelTrain=[]
            dataTestValidation=[]
            labelTestValidation=[]

            with open('logcontrast\c'+str(path1)+'\c'+str(path2)+'\s' + subjects + '.npy', 'rb') as file:
                sub = np.load(file, allow_pickle=True)
                for i in range (0,sub.shape[0]):
                    if i in random_list_Train:
                        dataTrain.append(np.ravel(sub[i][0]))
                        labelTrain.append(sub[i][1])
                    if i in random_list_Test:
                        dataTest.append(np.ravel(sub[i][0]))
                        labelTest.append(sub[i][1])
                    if i in random_list_Validation:
                        dataValidation.append(np.ravel(sub[i][0]))
                        labelValidation.append(sub[i][1])

            #print("subjects:", subjects)

            #print('loading data...')
            X  = np.array(dataTrain)
            Y  = np.array(labelTrain)
            X = normalize(X)



            M = np.array(dataTest)
            N = np.array(labelTest)
            M = normalize(M)



            O = np.array(dataValidation)
            P = np.array(labelValidation)
            O = normalize(O)
            np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\data_testing'+str(subjects)+'.npy', np.array(dataTest), allow_pickle=True, fix_imports=True)
            np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\label_testing'+str(subjects)+'.npy', np.array(labelTest), allow_pickle=True, fix_imports=True)
            np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\data_validationing'+str(subjects)+'.npy', np.array(dataValidation), allow_pickle=True, fix_imports=True)
            np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\label_validationing'+str(subjects)+'.npy', np.array(labelValidation), allow_pickle=True, fix_imports=True)
            np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\data_training'+str(subjects)+'.npy', np.array(dataTrain), allow_pickle=True, fix_imports=True)
            np.save('logcontrast\c'+str(path1)+'\c'+str(path2)+'\label_training'+str(subjects)+'.npy', np.array(labelTrain), allow_pickle=True, fix_imports=True)


            train_size = np.array(X).shape[0]
            #print(np.array(X).shape[0],"------train_size------",np.array(X).shape[1])


            #print("-----------svm1分类----------")
            SV = svm.SVC(kernel='rbf')
            SV.fit(X, Y)
            acc_matrix=SV.score(M, N)
            acc_v1.append(acc_matrix)

            #print(acc_matrix)


        acc_v_mean1 = np.mean(acc_v1)
        #for i in range(len(subjectList)):
            #print("subjects[",i,"]")
            #print("acc_v_mean:",acc_v1[i])
        print("------------")
        print("path:",path1,path2)
        print("acc_v_mean:",acc_v_mean1)

------------
path: waven 0
acc_v_mean: 0.8790625
------------
path: rir 0
acc_v_mean: 0.85421875
------------
path: psi 0
acc_v_mean: 0.821640625
------------
path: waven 1
acc_v_mean: 0.886015625
------------
path: rir 1
acc_v_mean: 0.859140625
------------
path: psi 1
acc_v_mean: 0.83078125


In [180]:
def FFT_Processing_subbasefeature_selectchannel_PSE(sub, channel,band, window_size, step_size, sample_rate, path1,path2):
    meta_de = []
    with open('data\s' + subjects + '.dat', 'rb') as file:
        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1
        for i in range (0,40):
            data = subject["data"][i]
            labels = subject["labels"][i]
            nlabel = 0
            if labels[path2]>=5:
                nlabel = 1
            start = 384
            allChannelbase=[]#保存32个通道的特征基线
            for j in channel:
                base = np.array(data[j][0:384]).reshape(3,128)
                basefeature=[]#3s每个特征的特征基线
                for k in range(3):
                    #Power_Ratio= mybin_power(base[k], band, sample_rate)
                    #Y =pyeeg.spectral_entropy(base[k], band, sample_rate,Power_Ratio)
                    Y = bin_power(base[k], band, sample_rate)
                    basefeature.append(Y[1]*np.log2(Y[1]))
                basevalue=np.array(basefeature).mean(axis=0)
                allChannelbase.append(basevalue)
            #print(np.array(allChannelbase).shape)
            while start + window_size <= 8064: # 使用窗口将每个通道的信号分为464份，可以产生464个样本（去掉前3秒的数据）
                meta_array = []
                meta_data = [] #meta vector for analysis
                num=0
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    #Power_Ratio= mybin_power(X, band, sample_rate)
                    #Y =pyeeg.spectral_entropy(X, band, sample_rate,Power_Ratio)
                    #meta_data = meta_data +[Y-allChannelbase[num]]
                    meta_data = meta_data + list((Y[1]*np.log2(Y[1]))-allChannelbase[num])


                    num=num+1
                    #print(np.array(meta_data).shape)
                
                meta_array.append(np.array(meta_data))
                #print(meta_array)
                meta_array.append(nlabel)

                meta_de.append(np.array(meta_array))      
                   
                start = start + step_size
                
        meta_de = np.array(meta_de)
        np.save('log2p\c'+str(path1)+'\c'+str(path2)+'\s' + sub, meta_de, allow_pickle=True, fix_imports=True) 

In [181]:
for path2 in range(0,2):
    for subjects in subjectList:
        FFT_Processing_subbasefeature_selectchannel_PSE (subjects, channel, band, window_size, step_size, sample_rate, 'pse', path2)
random_list_Test=[]
random_list_Validation=[]
random_list_Train=[]
with open('log1\list_train'+'.npy', 'rb') as fileTrain:
    random_list_Train  = np.load(fileTrain)
with open('log1\list_test'+'.npy', 'rb') as fileTrain:
    random_list_Test  = np.load(fileTrain)
with open('log1\list_validation'+'.npy', 'rb') as fileTrain:
    random_list_Validation  = np.load(fileTrain)
#print("测试集",random_list_Test)
#print("验证集",random_list_Validation)
#print("训练集",random_list_Train)
namelist=['pse']

for path2 in range(0,2):
    for path1 in namelist:
        acc_v1 = []
        for subjects in subjectList:
            dataTest=[]
            labelTest=[]
            dataValidation=[]
            labelValidation=[]
            dataTrain=[]
            labelTrain=[]
            dataTestValidation=[]
            labelTestValidation=[]

            with open('log2p\c'+str(path1)+'\c'+str(path2)+'\s' + subjects + '.npy', 'rb') as file:
                sub = np.load(file, allow_pickle=True)
                for i in range (0,sub.shape[0]):
                    if i in random_list_Train:
                        dataTrain.append(np.ravel(sub[i][0]))
                        labelTrain.append(sub[i][1])
                    if i in random_list_Test:
                        dataTest.append(np.ravel(sub[i][0]))
                        labelTest.append(sub[i][1])
                    if i in random_list_Validation:
                        dataValidation.append(np.ravel(sub[i][0]))
                        labelValidation.append(sub[i][1])

            #print("subjects:", subjects)

            #print('loading data...')
            X  = np.array(dataTrain)
            Y  = np.array(labelTrain)
            X = normalize(X)



            M = np.array(dataTest)
            N = np.array(labelTest)
            M = normalize(M)



            O = np.array(dataValidation)
            P = np.array(labelValidation)
            O = normalize(O)
            np.save('log2p\c'+str(path1)+'\c'+str(path2)+'\data_testing'+str(subjects)+'.npy', np.array(dataTest), allow_pickle=True, fix_imports=True)
            np.save('log2p\c'+str(path1)+'\c'+str(path2)+'\label_testing'+str(subjects)+'.npy', np.array(labelTest), allow_pickle=True, fix_imports=True)
            np.save('log2p\c'+str(path1)+'\c'+str(path2)+'\data_validationing'+str(subjects)+'.npy', np.array(dataValidation), allow_pickle=True, fix_imports=True)
            np.save('log2p\c'+str(path1)+'\c'+str(path2)+'\label_validationing'+str(subjects)+'.npy', np.array(labelValidation), allow_pickle=True, fix_imports=True)
            np.save('log2p\c'+str(path1)+'\c'+str(path2)+'\data_training'+str(subjects)+'.npy', np.array(dataTrain), allow_pickle=True, fix_imports=True)
            np.save('log2p\c'+str(path1)+'\c'+str(path2)+'\label_training'+str(subjects)+'.npy', np.array(labelTrain), allow_pickle=True, fix_imports=True)


            train_size = np.array(X).shape[0]
            #print(np.array(X).shape[0],"------train_size------",np.array(X).shape[1])


            #print("-----------svm1分类----------")
            SV = svm.SVC(kernel='rbf')
            SV.fit(X, Y)
            acc_matrix=SV.score(M, N)
            acc_v1.append(acc_matrix)

            #print(acc_matrix)


        acc_v_mean1 = np.mean(acc_v1)
        #for i in range(len(subjectList)):
            #print("subjects[",i,"]")
            #print("acc_v_mean:",acc_v1[i])
        print("------------")
        print("path:",path1,path2)
        print("acc_v_mean:",acc_v_mean1)

------------
path: pse 0
acc_v_mean: 0.90796875
------------
path: pse 1
acc_v_mean: 0.9117968750000001
